In [47]:
import pandas as pd
import numpy as np

In [48]:
cd D:\HKUST\data mining and knowledge discovery\Project\work2

D:\HKUST\data mining and knowledge discovery\Project\work2


In [49]:
aq_17_18 = pd.read_csv('airQuality_201701-201801.csv')
aq_1802_1803 = pd.read_csv('airQuality_201802-201803.csv')
aq_1804 = pd.read_csv('aiqQuality_201804.csv')
ow_1701_1801_full = pd.read_csv('ow_1701_1801_full.csv')

In [50]:
# Use spatial relations to fill in the neighborhood: find the nearest neighbor point data as a substitute; 
# if the same is missing, expand the range to find the nearest neighbor
# Use time to fill the missing value: take the air quality data near the time of the missing position as the substitute
def get_map():
    air_station_position = pd.read_csv('air_station_position.csv')
    aq_stationId = list(air_station_position['stationId'])
    aq_longitude = list(air_station_position['longitude'])
    aq_latitude = list(air_station_position['latitude'])
    aq_type = list(air_station_position['type'])
    n_pos = 35

    
    # Caculate the distance
    distance = []
    neighbor = []
    for i in range(13):
        temp = []
        index = []
        for j in range(n_pos):
            index.append(j)
        del index[i]
        for k in range(len(index)):
            temp.append(pow(aq_longitude[i] - aq_longitude[index[k]],2) + pow(aq_latitude[i] - aq_latitude[index[k]],2))
        distance.append(temp)
        # identify the nearest neighbor
        aq_distance = sorted(temp)
        aq_neighbor = []
        for j in range(12):
            t_index = temp.index(aq_distance[j])
            if t_index < i:
                aq_neighbor.append(aq_stationId[t_index])
            else:
                aq_neighbor.append(aq_stationId[t_index + 1])
            neighbor.append(aq_neighbor)

    # get the nearest station pair
    best_pair8full = {}
    for i in range(n_pos):
        for j in range(12):
            best_pair4full.setdefault(aq_stationId[i],[]).append(neighbor[i][j])
    return best_pair4full
best_pair12full = get_map()

In [51]:
best_pair12full

{'dongsi_aq': ['qianmen_aq',
  'tiantan_aq',
  'qianmen_aq',
  'tiantan_aq',
  'nongzhanguan_aq',
  'aotizhongxin_aq',
  'yongdingmennei_aq',
  'dongsihuan_aq',
  'xizhimenbei_aq',
  'guanyuan_aq',
  'wanshouxigong_aq',
  'nansanhuan_aq',
  'wanliu_aq',
  'fengtaihuayuan_aq',
  'qianmen_aq',
  'tiantan_aq',
  'nongzhanguan_aq',
  'aotizhongxin_aq',
  'yongdingmennei_aq',
  'dongsihuan_aq',
  'xizhimenbei_aq',
  'guanyuan_aq',
  'wanshouxigong_aq',
  'nansanhuan_aq',
  'wanliu_aq',
  'fengtaihuayuan_aq'],
 'tiantan_aq': ['qianmen_aq',
  'tiantan_aq',
  'qianmen_aq',
  'tiantan_aq',
  'nongzhanguan_aq',
  'aotizhongxin_aq',
  'yongdingmennei_aq',
  'dongsihuan_aq',
  'xizhimenbei_aq',
  'guanyuan_aq',
  'wanshouxigong_aq',
  'nansanhuan_aq',
  'wanliu_aq',
  'fengtaihuayuan_aq',
  'qianmen_aq',
  'tiantan_aq',
  'nongzhanguan_aq',
  'aotizhongxin_aq',
  'yongdingmennei_aq',
  'dongsihuan_aq',
  'xizhimenbei_aq',
  'guanyuan_aq',
  'wanshouxigong_aq',
  'nansanhuan_aq',
  'wanliu_aq',
  '

In [52]:
from tqdm import tqdm
import datetime

aq_map = {}
aq_id = list(aq_17_18['stationId'])
aq_time = list(aq_17_18['utc_time'])
aq_pm25 = list(aq_17_18['PM2.5'])
aq_pm10 = list(aq_17_18['PM10'])
aq_no2 = list(aq_17_18['NO2'])
aq_co = list(aq_17_18['CO'])
aq_o3 = list(aq_17_18['O3'])
aq_so2 = list(aq_17_18['SO2'])
data2full_index = [aq_pm25,aq_pm10,aq_no2,aq_co,aq_o3,aq_so2] 

for i in range(len(aq_id)):
    key = str(aq_id[i]) + str(aq_time[i]) 
    for each in data2full_index:
        aq_map.setdefault(key,[]).append(each[i])

# fill
aq_pm25,aq_pm10,aq_no2,aq_co,aq_o3,aq_so2 = list(),list(),list(),list(),list(),list()
aq_index = [aq_pm25,aq_pm10,aq_no2,aq_co,aq_o3,aq_so2]
avg_fillna = [58.785570,88.059259,45.792457,0.960677,55.692755,8.981003]

for i in tqdm(range(len(aq_id))):
    for j in range(len(aq_index)):
        # No missing value
        if not np.isnan(data2full_index[j][i]):
            aq_index[j].append(data2full_index[j][i])
        # fill missing value by using the nearest neighbor
        elif (str(best_pair8full[str(aq_id[i])][0]) + str(aq_time[i])) in aq_map:
            if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][0]) + str(aq_time[i])][j]):
                aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][0]) + str(aq_time[i])][j])
            elif (str(best_pair8full[str(aq_id[i])][1]) + str(aq_time[i])) in aq_map:
                if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][1]) + str(aq_time[i])][j]):
                    aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][1]) + str(aq_time[i])][j])
                elif (str(best_pair8full[str(aq_id[i])][2]) + str(aq_time[i])) in aq_map:
                    if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][2]) + str(aq_time[i])][j]):
                        aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][2]) + str(aq_time[i])][j])
                    elif (str(best_pair8full[str(aq_id[i])][3]) + str(aq_time[i])) in aq_map:
                        if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][3]) + str(aq_time[i])][j]):
                            aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][3]) + str(aq_time[i])][j])
                        elif (str(best_pair8full[str(aq_id[i])][4]) + str(aq_time[i])) in aq_map:
                            if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][4]) + str(aq_time[i])][j]):
                                aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][4]) + str(aq_time[i])][j])
                            elif (str(best_pair8full[str(aq_id[i])][5]) + str(aq_time[i])) in aq_map:
                                if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][5]) + str(aq_time[i])][j]):
                                    aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][5]) + str(aq_time[i])][j])
                                elif (str(best_pair8full[str(aq_id[i])][6]) + str(aq_time[i])) in aq_map:
                                    if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][6]) + str(aq_time[i])][j]):
                                        aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][6]) + str(aq_time[i])][j])
                                    elif (str(best_pair8full[str(aq_id[i])][7]) + str(aq_time[i])) in aq_map:
                                        if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][7]) + str(aq_time[i])][j]):
                                            aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][7]) + str(aq_time[i])][j])
                                        elif (str(best_pair8full[str(aq_id[i])][7]) + str(aq_time[i])) in aq_map:
                                            if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][8]) + str(aq_time[i])][j]):
                                                aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][8]) + str(aq_time[i])][j])
                                            elif (str(best_pair8full[str(aq_id[i])][9]) + str(aq_time[i])) in aq_map:
                                                if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][9]) + str(aq_time[i])][j]):
                                                    aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][9]) + str(aq_time[i])][j])
                                                elif (str(best_pair8full[str(aq_id[i])][10]) + str(aq_time[i])) in aq_map:
                                                    if not np.isnan(aq_map[str(best_pair8full[str(aq_id[i])][10]) + str(aq_time[i])][j]):
                                                        aq_index[j].append(aq_map[str(best_pair8full[str(aq_id[i])][10]) + str(aq_time[i])][j])
                                                    else:
                                                        aq_index[j].append(avg_fillna[j])  
                                                else:
                                                    aq_index[j].append(avg_fillna[j])    
                                            else:
                                                aq_index[j].append(avg_fillna[j])
                                        else:
                                            aq_index[j].append(avg_fillna[j])
                                    else:
                                        aq_index[j].append(avg_fillna[j])
                                else:
                                    aq_index[j].append(avg_fillna[j])
                            else:
                                aq_index[j].append(avg_fillna[j])
                        else:
                            aq_index[j].append(avg_fillna[j])
                    else:
                        aq_index[j].append(avg_fillna[j])
                else:
                    aq_index[j].append(avg_fillna[j])
            else:
                aq_index[j].append(avg_fillna[j])
        else:
            aq_index[j].append(avg_fillna[j])

100%|███████████████████████████████████████████████████████████████████████| 311010/311010 [00:04<00:00, 68069.48it/s]


In [53]:
aq_index[1]

[467.0,
 443.0,
 467.0,
 484.0,
 520.0,
 459.0,
 428.0,
 88.059259,
 88.059259,
 88.059259,
 88.059259,
 220.0,
 319.0,
 479.0,
 247.0,
 314.0,
 211.0,
 291.0,
 88.059259,
 88.059259,
 88.059259,
 88.059259,
 88.059259,
 223.0,
 223.0,
 266.0,
 295.0,
 358.0,
 400.0,
 436.0,
 401.0,
 436.0,
 419.0,
 361.0,
 318.0,
 340.0,
 392.0,
 396.0,
 383.0,
 323.0,
 276.0,
 461.0,
 334.0,
 417.0,
 356.0,
 420.0,
 448.0,
 486.0,
 642.0,
 476.0,
 493.0,
 502.0,
 744.0,
 504.0,
 539.0,
 561.0,
 506.0,
 632.0,
 601.0,
 342.0,
 469.0,
 554.0,
 588.0,
 317.0,
 393.0,
 331.0,
 338.0,
 360.0,
 340.0,
 399.0,
 368.0,
 406.0,
 396.0,
 333.0,
 88.059259,
 88.059259,
 88.059259,
 88.059259,
 192.0,
 360.0,
 401.0,
 324.0,
 362.0,
 355.0,
 303.0,
 272.0,
 88.059259,
 88.059259,
 88.059259,
 88.059259,
 208.0,
 226.0,
 207.0,
 199.0,
 202.0,
 199.0,
 214.0,
 210.0,
 214.0,
 226.0,
 204.0,
 175.0,
 181.0,
 178.0,
 180.0,
 191.0,
 197.0,
 214.0,
 229.0,
 249.0,
 211.0,
 247.0,
 264.0,
 234.0,
 186.0,
 194.0,
 207

In [54]:
for i in range(len(aq_index[1])):
    if aq_index[1][i] == 88.059259:
        aq_index[1][i] = 1.8079*aq_index[0][i]+17.7440

In [55]:
aq_full = []
index = [aq_id,aq_time,aq_pm25,aq_pm10,aq_no2,aq_co,aq_o3,aq_so2]

for each in index:
    aq_full.append(each)

c_index = ['stationId','utc_time','PM2.5','PM10','NO2','CO','O3','SO2']

df_aq_full = pd.DataFrame(np.transpose(aq_full) ,index = None, columns = c_index)
df_aq_full.to_csv('air_quality_full.csv')
df_aq_full.tail()

,stationId,utc_time,PM2.5,PM10,NO2,CO,O3,SO2
311005,zhiwuyuan_aq,2018-01-31 11:00:00,36.0,80.0,74.0,0.9,23.0,9.0
311006,zhiwuyuan_aq,2018-01-31 12:00:00,52.0,89.0,86.0,1.2,14.0,11.0
311007,zhiwuyuan_aq,2018-01-31 13:00:00,54.0,88.0,69.0,1.1,25.0,13.0
311008,zhiwuyuan_aq,2018-01-31 14:00:00,51.0,89.0,79.0,1.0,15.0,12.0
311009,zhiwuyuan_aq,2018-01-31 15:00:00,47.0,87.0,89.0,1.0,6.0,12.0


In [56]:
def get_missing_data_table(data):
    total = data.isnull().sum()
    percentage = data.isnull().sum() / data.isnull().count()
    missing_data = pd.concat([total,percentage],axis = 'columns',keys = ['TOTAL','PERCENTAGE'])
    return missing_data.sort_index(ascending = True)

In [57]:
get_missing_data_table(df_aq_full)

,TOTAL,PERCENTAGE
CO,0,0.0
NO2,0,0.0
O3,0,0.0
PM10,0,0.0
PM2.5,0,0.0
SO2,0,0.0
stationId,0,0.0
utc_time,0,0.0
